In [1]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider, Band
from bokeh.io import output_file, show
import pandas as pd
import numpy as np

from bokeh.plotting import figure

import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (14,10)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
#Settings for plotting
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)

%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

In [2]:
datafilename = '../model_results_spread_prediction_var_vacc.csv'
df = pd.read_csv(datafilename)
df.End =  pd.to_datetime(df.End,format='%Y-%m-%d')

In [3]:
# curdf = df[df.Age == '20-29']
df['RT2'] = np.round(np.multiply(df.RT,10)).astype(int)
df

,Unnamed: 0,Age,CaseHospRate,Population,Start,End,RT,Count,CountCumul,vacc1,PctCumul,HospCumul,RT2
0,0,10-19,0.01,273589,2021-05-18,2021-05-18,0.9,6.860000e+02,0.000000,0.4,0.000000,0.000000,9
1,1,10-19,0.01,273589,2021-05-18,2021-05-25,0.9,6.391477e+02,639.147698,0.4,0.002336,6.391477,9
2,2,10-19,0.01,273589,2021-05-18,2021-06-01,0.9,5.921772e+02,1231.324890,0.4,0.004501,12.313249,9
3,3,10-19,0.01,273589,2021-05-18,2021-06-08,0.9,5.387920e+02,1770.116848,0.4,0.006470,17.701168,9
4,4,10-19,0.01,273589,2021-05-18,2021-06-15,0.9,4.422557e+02,2212.372554,0.4,0.008086,22.123726,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19597,19597,90+,0.40,45458,2021-05-18,2021-09-14,1.7,4.191857e-12,1.653221,0.9,0.000036,0.661288,17
19598,19598,90+,0.40,45458,2021-05-18,2021-09-21,1.7,8.005930e-13,1.653221,0.9,0.000036,0.661288,17
19599,19599,90+,0.40,45458,2021-05-18,2021-09-28,1.7,1.529034e-13,1.653221,0.9,0.000036,0.661288,17
19600,19600,90+,0.40,45458,2021-05-18,2021-10-05,1.7,2.920266e-14,1.653221,0.9,0.000036,0.661288,17


In [4]:
curdf = df[df.vacc1 == 0.75]

,Unnamed: 0,Age,CaseHospRate,Population,Start,End,RT,Count,CountCumul,vacc1,PctCumul,HospCumul,RT2
154,154,10-19,0.01,273589,2021-05-18,2021-05-18,0.9,6.860000e+02,0.000000,0.75,0.000000,0.000000,9
155,155,10-19,0.01,273589,2021-05-18,2021-05-25,0.9,6.391477e+02,639.147698,0.75,0.002336,6.391477,9
156,156,10-19,0.01,273589,2021-05-18,2021-06-01,0.9,5.921772e+02,1231.324890,0.75,0.004501,12.313249,9
157,157,10-19,0.01,273589,2021-05-18,2021-06-08,0.9,5.326283e+02,1763.953149,0.75,0.006447,17.639531,9
158,158,10-19,0.01,273589,2021-05-18,2021-06-15,0.9,3.897356e+02,2153.688770,0.75,0.007872,21.536888,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19531,19531,90+,0.40,45458,2021-05-18,2021-09-14,1.7,4.191857e-12,1.653221,0.75,0.000036,0.661288,17
19532,19532,90+,0.40,45458,2021-05-18,2021-09-21,1.7,8.005930e-13,1.653221,0.75,0.000036,0.661288,17
19533,19533,90+,0.40,45458,2021-05-18,2021-09-28,1.7,1.529034e-13,1.653221,0.75,0.000036,0.661288,17
19534,19534,90+,0.40,45458,2021-05-18,2021-10-05,1.7,2.920266e-14,1.653221,0.75,0.000036,0.661288,17


In [5]:

# curdf['x'] = np.arange(0,len(curdf))
# curdf['y'] = np.arange(0,len(curdf))

casedf = pd.DataFrame()

casedf['Date'] = curdf.End.unique().astype('datetime64[D]')
numVals = len(curdf.End.unique())
firstWeek = 20
weekNums = np.arange(firstWeek,firstWeek+numVals)
casedf['Week'] = weekNums

 


allAges = df.Age.unique()

allRTs = df.RT2.unique()


casedf['Total'] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values.shape)

for curRT in allRTs:
    curColName = 'Total_'+str(curRT)
    casedf[curColName] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values.shape)

for curAge in allAges:
    casedf[curAge] = curdf[(curdf.Age == curAge) & (curdf.RT2 == 11)].Count.values 
    casedf['Total'] += curdf[(curdf.Age == curAge) & (curdf.RT2 == 11)].Count.values 

    for curRT in allRTs:
        curColName = curAge+'_'+str(curRT)
        casedf[curColName] = curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Count.values 
        casedf['Total_'+str(curRT)] += curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Count.values 


casedf


,Date,Week,Total,Total_9,Total_10,Total_11,Total_12,Total_13,Total_14,Total_15,...,90+,90+_9,90+_10,90+_11,90+_12,90+_13,90+_14,90+_15,90+_16,90+_17
0,2021-05-18,20,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,...,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00
1,2021-05-25,21,5737.029927,5013.857624,5381.395161,5737.029927,6082.182325,6417.999902,6745.426619,7065.250857,...,9.984850e-01,8.726226e-01,9.365896e-01,9.984850e-01,1.058556e+00,1.117003e+00,1.173989e+00,1.229651e+00,1.284107e+00,1.337455e+00
2,2021-06-01,22,5712.539396,4363.138230,5026.257629,5712.539396,6420.573872,7149.149679,7897.212454,8663.834553,...,1.423780e-01,1.087458e-01,1.252732e-01,1.423780e-01,1.600248e-01,1.781837e-01,1.968282e-01,2.159353e-01,2.354845e-01,2.554573e-01
3,2021-06-08,23,5653.534757,3775.813770,4667.231832,5653.534757,6734.795493,7911.077371,9182.435724,10548.919136,...,2.030129e-02,1.355130e-02,1.675515e-02,2.030129e-02,2.419020e-02,2.842229e-02,3.299795e-02,3.791756e-02,4.318144e-02,4.878991e-02
4,2021-06-15,24,5384.119727,3147.864843,4172.794557,5384.119727,6793.940046,8413.934336,10255.400844,12329.290436,...,2.894687e-03,1.688679e-03,2.240968e-03,2.894687e-03,3.656690e-03,4.533634e-03,5.532006e-03,6.658136e-03,7.918213e-03,9.318299e-03
5,2021-06-22,25,4983.133105,2554.715157,3628.860219,4983.133105,6654.145389,8679.174464,11096.037054,13942.971774,...,4.127426e-04,2.104325e-04,2.997248e-04,4.127426e-04,5.527597e-04,7.231583e-04,9.274225e-04,1.169134e-03,1.451966e-03,1.779681e-03
6,2021-06-29,26,4523.770779,2038.116440,3099.122675,4523.770779,6383.696702,8755.367797,11719.512683,15360.498523,...,5.885140e-05,2.622278e-05,4.008756e-05,5.885140e-05,8.355734e-05,1.153507e-04,1.554793e-04,2.052937e-04,2.662474e-04,3.398972e-04
7,2021-07-06,27,3949.010265,1567.726965,2548.872105,3949.010265,5878.061610,8458.000279,11821.301708,16108.851252,...,8.391398e-06,3.267718e-06,5.361628e-06,8.391398e-06,1.263086e-05,1.839954e-05,2.606558e-05,3.604848e-05,4.882187e-05,6.491618e-05
8,2021-07-13,28,3161.210499,1109.179955,1925.713894,3161.210499,4951.901078,7453.195746,10835.814736,15279.771078,...,1.196498e-06,4.072024e-07,7.171065e-07,1.196498e-06,1.909330e-06,2.934903e-06,4.369806e-06,6.329920e-06,8.952481e-06,1.239819e-05
9,2021-07-20,29,2273.527257,706.895497,1309.194631,2273.527257,3739.380464,5868.514492,8837.013750,12822.022065,...,1.706041e-07,5.074300e-08,9.591149e-08,1.706041e-07,2.886218e-07,4.681451e-07,7.325833e-07,1.111500e-06,1.641619e-06,2.367902e-06


In [22]:

# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)

plot = figure(plot_width=800, plot_height=400)
plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color='#109000')
plot.line('Week', '20-29', source=source, line_width=3, line_alpha=0.6, color='#208000')
plot.line('Week', '30-39', source=source, line_width=3, line_alpha=0.6, color='#307000')
plot.line('Week', '40-49', source=source, line_width=3, line_alpha=0.6, color='#406000')
plot.line('Week', '50-59', source=source, line_width=3, line_alpha=0.6, color='#505000')
plot.line('Week', '60-69', source=source, line_width=3, line_alpha=0.6, color='#604000')
plot.line('Week', '70-79', source=source, line_width=3, line_alpha=0.6, color='#703000')
plot.line('Week', '80-89', source=source, line_width=3, line_alpha=0.6, color='#802000')
plot.line('Week',   '90+', source=source, line_width=3, line_alpha=0.6, color='#901000')
plot.line('Week', 'Total', source=source, line_width=3, line_alpha=0.6,color='black')

# plot.circle('Week', '10-19', source=source, color='#108000')
# plot.circle('Week', '20-29', source=source, color='#208000')
# plot.circle('Week', '30-39', source=source, color='#308000')
# plot.circle('Week', '40-49', source=source, color='#408000')
# plot.circle('Week', '50-59', source=source, color='#508000')
# plot.circle('Week', '60-69', source=source, color='#608000')
# plot.circle('Week', '70-79', source=source, color='#708000')
# plot.circle('Week', '80-89', source=source, color='#808000')
# plot.circle('Week',   '90+', source=source, color='#908000')
# plot.circle('Week', 'Total', source=source,color='black')
# plot.line('Week', 'Total', source=source, line_width=3, line_alpha=0.6)


plot.varea('Week', 0 ,'10-19', source=source, fill_color='#109000')
plot.varea('Week','10-19', '20-29', source=source, color='#208000')
plot.varea('Week','20-29', '30-39', source=source, color='#307000')
plot.varea('Week','30-39', '40-49', source=source, color='#406000')
plot.varea('Week','40-49', '50-59', source=source, color='#505000')
plot.varea('Week','50-59', '60-69', source=source, color='#604000')
plot.varea('Week','60-69', '70-79', source=source, color='#703000')
plot.varea('Week','70-79', '80-89', source=source, color='#802000')
plot.varea('Week','80-89',   '90+', source=source, color='#901000')

# slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")
minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.1, step=.1, title="RT")

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Just numbers
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i]
        y3[i] = data[curName3][i]
        y4[i] = data[curName4][i]
        y5[i] = data[curName5][i]
        y6[i] = data[curName6][i]
        y7[i] = data[curName7][i]
        y8[i] = data[curName8][i]
        y9[i] = data[curName9][i]
        yT[i] = data[curNameT][i]
    }
    
    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)

output_file("plot.html")

show(column(plot,slider))

In [6]:
# curdf['RT2'] = np.round(np.multiply(curdf.RT,10)).astype(int)

In [91]:

# # x = [x*0.005 for x in range(0, 201)] 

# # source = ColumnDataSource(data=dict(x=x, y=x))
# # source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
# source = ColumnDataSource(curdf)

# plot = figure(plot_width=400, plot_height=400)
# plot.line('x', 'Count', source=source, line_width=3, line_alpha=0.6)

# # slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")
# minRT = 0.9
# maxRT = 1.7
# slider = Slider(start=minRT, end=maxRT, value=1.1, step=.1, title="RT")

# update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
#     var data = source.data;
#     var f = slider.value;
#     var age = data['Age']
#     var RT = data['RT2']
#     var End = data['End']
#     var Count = data['Count']

#     var x = data['x']
#     var y = data['y']

#     console.log(Count)
#     for (var i = 0; i < RT.length; i++) {
#         y[i] = 6
#         Count[i] = 0
        
#         if (RT[i] == Math.round(f*10)){
#             y[i] = data['Count'][i]
#             console.log(data['Count'][i])
#             Count[i] = data['Count'][i]

#             // End.push(data['End'][i])
#             // Count.push(data['Count'][i])
#         }

#     }

#     //for (var i = 0; i < x.length; i++) {
#         // y[i] = Math.pow(x[i], f)
#     //}
    
#     // necessary becasue we mutated source.data in-place
#     source.change.emit();
# """)
# slider.js_on_change('value', update_curve)

# output_file("plot.html")

# show(column(slider, plot))

In [ ]:

# testdf['Total'] = curdf[(curdf.RT2 == 11)].Count.values 
# for curRT in allRTs:
#     curColName = 'Total_'+str(curRT)
#     testdf[curColName] = curdf[(curdf.RT2 == curRT)].Count.values 

# testdf['20_RT10'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 10)].Count.values
# testdf['20_RT11'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values
# testdf['20_RT12'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values
# testdf['20_RT13'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values
# testdf['20_RT14'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 14)].Count.values
# testdf['RT11_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values
# testdf['RT12_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values
# testdf['RT13_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values
# testdf['RT14_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 14)].Count.values

    # if (Math.round(f*10) == 11){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT11_20'][i]
    #     }
    # }
    # if (Math.round(f*10) == 12){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT12_20'][i]
    #     }
    # }
    # if (Math.round(f*10) == 13){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT13_20'][i]
    #     }
    # }
    
